In [246]:
using DataFrames, CSV, JuMP, Gurobi, Random, LibPQ, Statistics
using HTTP, JSON

In [ ]:
#TODO: August 17, August 24 ADP data
#TODO: figure out how to rate the players that are never "starters"

In [364]:
function initialize_data()
    #data = projections for each player
    data = DataFrame(CSV.File("dynasty_data.csv"))
    #fill missing with 0 in julia
    replace!(data.ADP_SF, missing => 500)
    data.Name .= data.FirstName .* " " .* data.LastName
    sort!(data, :ADP_SF)
    data

    player_to_index = Dict()
    #iterate thorugh each row in the data
    for i in 1:size(data)[1]
        #get the player name
        player = data[i, :Name]
        #if the player is not in the dictionary, add them
        player_to_index[player] = i
    end

    qbs = Set()
    rbs = Set()
    wrs = Set()
    tes = Set()
    flex = Set()
    names = Set()

    for player in eachrow(data)
        push!(names, player.Name)
        if player.Position == "QB"
            push!(qbs, player.Name)
        elseif player.Position == "RB"
            push!(rbs, player.Name)
            push!(flex, player.Name)
        elseif player.Position == "WR"
            push!(wrs, player.Name)
            push!(flex, player.Name)
        elseif player.Position == "TE"
            push!(tes, player.Name)
        end
    end
    return data, player_to_index, qbs, rbs, wrs, tes, flex, names
end

initialize_data (generic function with 1 method)

In [298]:
function reset_draft()
    drafted = Dict()
    for i in range(1, 10)
        drafted[i] = []
    end
    return drafted
end

reset_draft (generic function with 1 method)

In [301]:
function just_drafted(player, team)
    if length(filter(:Name => n -> n == player, data)[!,1]) < 1
        return "Player not found"
    end
    if team < 1 || team > 10
        return "Team not found"
    end
    for i in 1:10
        if player in drafted[i]
            return "Player already drafted"
        end
    end
    push!(drafted[team], player)
    return "Team " * string(team) * " drafted " * player * "!"
end

just_drafted (generic function with 1 method)

In [397]:
function players_drafted_opps(team)
    drafted_opps = DataFrame(Name = data.Name, Drafted = fill(false, size(data)[1]))
    for i in 1:10
        if i == team
            continue
        else
            for player in drafted[i]
                if player == "QB" || player == "RB" || player == "WR" || player == "TE"
                    continue
                end
                drafted_opps[player_to_index[player], "Drafted"] = true
            end
        end
    end
    return drafted_opps
end

players_drafted_opps (generic function with 1 method)

In [211]:
function draft_board()
    board = DataFrame(T1 = String[], T2 = String[], T3 = String[], T4 = String[], T5 = String[], T6 = String[], T7 = String[], T8 = String[], T9 = String[], T10 = String[])
    for i in 1:26
        push!(board, ["", "", "", "", "", "", "", "", "", ""])
    end
    for team in 1:10
        num = 1
        for player in drafted[team]
            board[team, num] = player
            num += 1
        end
    end
    return board
end

draft_board (generic function with 1 method)

In [353]:
function view_roster(team)
    roster = DataFrame(Position = String[], Player = String[])
    push!(roster, ["QB1", ""])
    push!(roster, ["QB2", ""])
    push!(roster, ["RB1", ""])
    push!(roster, ["RB2", ""])
    push!(roster, ["WR1", ""])
    push!(roster, ["WR2", ""])
    push!(roster, ["TE", ""])
    push!(roster, ["FLEX", ""])
    push!(roster, ["FLEX", ""])
    push!(roster, ["Bench", ""])
    push!(roster, ["Bench", ""])
    push!(roster, ["Bench", ""])
    push!(roster, ["Bench", ""])
    push!(roster, ["Bench", ""])
    push!(roster, ["Bench", ""])
    push!(roster, ["Bench", ""])
    push!(roster, ["Bench", ""])
    push!(roster, ["Bench", ""])
    push!(roster, ["Bench", ""])
    push!(roster, ["Bench", ""])
    push!(roster, ["Bench", ""])
    push!(roster, ["Bench", ""])
    push!(roster, ["Bench", ""])
    push!(roster, ["Bench", ""])
    push!(roster, ["Bench", ""])
    push!(roster, ["Bench", ""])

    qb_spots = [1, 2, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
    rb_spots = [3, 4, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
    wr_spots = [5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
    te_spots = [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]

    for player in drafted[team]
        if player == "QB"
            for spot in qb_spots
                if roster[spot, "Player"] == ""
                    roster[spot, "Player"] = player
                    break
                end
            end
            continue
        elseif player == "RB"
            for spot in rb_spots
                if roster[spot, "Player"] == ""
                    roster[spot, "Player"] = player
                    break
                end
            end
            continue
        elseif player == "WR"
            for spot in wr_spots
                if roster[spot, "Player"] == ""
                    roster[spot, "Player"] = player
                    break
                end
            end
            continue
        elseif player == "TE"
            for spot in te_spots
                if roster[spot, "Player"] == ""
                    roster[spot, "Player"] = player
                    break
                end
            end
            continue
        end
        player_pos = data[player_to_index[player], "Position"]
        if player_pos == "QB"
            for spot in qb_spots
                if roster[spot, "Player"] == ""
                    roster[spot, "Player"] = player
                    break
                end
            end
        elseif player_pos == "RB"
            for spot in rb_spots
                if roster[spot, "Player"] == ""
                    roster[spot, "Player"] = player
                    break
                end
            end
        elseif player_pos == "WR"
            for spot in wr_spots
                if roster[spot, "Player"] == ""
                    roster[spot, "Player"] = player
                    break
                end
            end
        elseif player_pos == "TE"
            for spot in te_spots
                if roster[spot, "Player"] == ""
                    roster[spot, "Player"] = player
                    break
                end
            end
        end
    end
    return roster
end

view_roster (generic function with 1 method)

In [337]:
function update_draft(id)
    url = "https://api.sleeper.app/v1/draft/" *id*"/picks"
    response = HTTP.get(url)
    response_body = String(response.body)
    parsed_json = JSON.parse(response_body)
    drafted = reset_draft()
    for pick in parsed_json
        team = Int64(pick["draft_slot"])
        position = pick["metadata"]["position"]
        name = pick["metadata"]["first_name"] * " " * pick["metadata"]["last_name"]
        name = replace(name, "." => "")
        if name in names
            push!(drafted[team], name)
        else
            push!(drafted[team], position)
        end
    end
    return drafted
end

update_draft (generic function with 1 method)

In [10]:
const GRB_ENV = Gurobi.Env(output_flag=0);

In [395]:
function dynasty_sf_model(team, pv, temp=[])
    # create model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(model, "TimeLimit", 300)

    # PARAMETERS
    P = size(data)[1] # num_players
    Y = 10 # num_years projecting

    min_qbs = 3
    max_qbs = 4
    min_rbs = 7
    max_rbs = 11
    min_wrs = 7
    max_wrs = 11
    min_tes = 2
    max_tes = 4

    starting_qbs_now = 2
    starting_rbs_now = 3
    starting_wrs_now = 3
    starting_flex_now = 10
    starting_tes_now = 1

    starting_qbs_mid = 2
    starting_rbs_mid = 2
    starting_wrs_mid = 2
    starting_flex_mid = 8
    starting_tes_mid = 1

    starting_qbs_fut = 1
    starting_flex_fut = 6
    starting_tes_fut = 1


    opps = players_drafted_opps(team)

    num_picked = sum(opps[i, "Drafted"] for i in 1:P) + length(drafted[team])

    picks = [team, 21-team]
    for i in 3:26
        push!(picks, picks[i-2]+20)
    end

    # VARIABLES
    @variable(model, x[i = 1:P], Bin) # whether or not player was drafted by desired team
    @variable(model, y[i = 1:P, j = 1:Y], Bin) # whether or not player is "starting" in year j on your roster

    # OBJECTIVE FUNCTION
    @objective(model, Max, sum(data[i, j+4]*y[i,j]*(pv^(j-1)) for i in 1:P, j in 1:Y))

    # CONSTRAINTS
    # roster must contain already drafted num_players
    @constraint(model, [i = drafted[team]], x[player_to_index[i]] == 1)

    # roster must not contain any opps
    @constraint(model, [i=1:P], x[i]+opps[i,"Drafted"] <= 1)

    # roster must not contain any temps
    @constraint(model, [i=1:length(temp)], x[player_to_index[temp[i]]] == 0)

    # selecting players in y only if on roster
    @constraint(model, [i = 1:P, j = 1:Y], y[i,j] <= x[i])

    # roster must contain 26 players (no kicker/DEF)
    @constraint(model, sum(x[i] for i in 1:P) <= 26)

    # positional constraints
    @constraint(model, sum(x[player_to_index[i]] for i in qbs) >= min_qbs)
    @constraint(model, sum(x[player_to_index[i]] for i in qbs) <= max_qbs)
    @constraint(model, sum(x[player_to_index[i]] for i in rbs) >= min_rbs)
    @constraint(model, sum(x[player_to_index[i]] for i in rbs) <= max_rbs)
    @constraint(model, sum(x[player_to_index[i]] for i in wrs) >= min_wrs)
    @constraint(model, sum(x[player_to_index[i]] for i in wrs) <= max_wrs)
    @constraint(model, sum(x[player_to_index[i]] for i in tes) >= min_tes)
    @constraint(model, sum(x[player_to_index[i]] for i in tes) <= max_tes)

    # starting constraints
    @constraint(model, [j = 1:2], sum(y[player_to_index[i],j] for i in qbs) == starting_qbs_now)
    @constraint(model, [j = 1:2], sum(y[player_to_index[i],j] for i in rbs) >= starting_rbs_now)
    @constraint(model, [j = 1:2], sum(y[player_to_index[i],j] for i in wrs) >= starting_wrs_now)
    @constraint(model, [j = 1:2], sum(y[player_to_index[i],j] for i in flex) == starting_flex_now)
    @constraint(model, [j = 1:2], sum(y[player_to_index[i],j] for i in tes) == starting_tes_now)

    @constraint(model, [j = 3:5], sum(y[player_to_index[i],j] for i in qbs) == starting_qbs_mid)
    @constraint(model, [j = 3:5], sum(y[player_to_index[i],j] for i in rbs) >= starting_rbs_mid)
    @constraint(model, [j = 3:5], sum(y[player_to_index[i],j] for i in wrs) >= starting_wrs_mid)
    @constraint(model, [j = 3:5], sum(y[player_to_index[i],j] for i in flex) == starting_flex_mid)
    @constraint(model, [j = 3:5], sum(y[player_to_index[i],j] for i in tes) == starting_tes_mid)

    @constraint(model, [j = 6:7], sum(y[player_to_index[i],j] for i in qbs) == starting_qbs_fut)
    @constraint(model, [j = 6:7], sum(y[player_to_index[i],j] for i in flex) == starting_flex_fut)
    @constraint(model, [j = 6:7], sum(y[player_to_index[i],j] for i in tes) == starting_tes_fut)

    @constraint(model, [j = 8:10], sum(y[i,j] for i in 1:P) >= 2)


    # DRAFT POSITION CONSTRAINTS
    #at least 1 pick from set of players after each round
    for pick in length(drafted[team])+1:26
        @constraint(model, sum(x[i] for i in (picks[pick]-sum(opps[j, "Drafted"] for j in picks[pick]:P)):P) >= 27-pick)
    end

    # OPTIMIZE
    # solvetime = @elapsed optimize!(model)
    optimize!(model)

    roster = DataFrame(Name = String[], Position = String[], Pick = Int64[], ADP = Float64[])
    count = 1
    for i in drafted[team]
        if i == "QB" || i == "RB" || i == "WR" || i == "TE"
            push!(roster, [i, i, picks[count], 500])
            continue
        end
        push!(roster, [data[player_to_index[i], "Name"], data[player_to_index[i], "Position"], picks[count], data[player_to_index[i], "ADP_SF"]])
        count += 1
    end
    for i in 1:P
        if value.(x[i]) == 1
            if data[i, "Name"] in drafted[team]
                continue
            else
                push!(roster, [data[i, "Name"], data[i, "Position"], picks[count], data[i, "ADP_SF"]])
                count += 1
            end
        end
    end
    next_pick = ""
    for i in 1:P
        if value.(x[i]) == 1
            if data[i, "Name"] in drafted[team]
                continue
            else
                next_pick = data[i, "Name"]
                break
            end
        end
    end
    println("Total VORP: " * string(round(objective_value(model), digits = 1)))
    return next_pick, roster
end

dynasty_sf_model (generic function with 3 methods)

In [ ]:
data, player_to_index, qbs, rbs, wrs, tes, flex, names = initialize_data()

In [431]:
drafted = update_draft("998069224713195520")

Dict{Any, Any} with 10 entries:
  5  => Any["Justin Jefferson", "Travis Kelce", "Cooper Kupp", "Josh Jacobs", "…
  4  => Any["Justin Herbert", "Breece Hall", "Tyreek Hill", "Drake London", "CJ…
  6  => Any["Ja'Marr Chase", "Garrett Wilson", "Jahmyr Gibbs", "Kenneth Walker"…
  7  => Any["Joe Burrow", "Amon-Ra St Brown", "Deshaun Watson", "Mark Andrews",…
  2  => Any["Patrick Mahomes", "Bijan Robinson", "Saquon Barkley", "Tua Tagovai…
  10 => Any["Justin Fields", "Christian McCaffrey", "Tee Higgins", "Kyle Pitts"…
  9  => Any["Trevor Lawrence", "CeeDee Lamb", "Dak Prescott", "Austin Ekeler", …
  8  => Any["Lamar Jackson", "Anthony Richardson", "Stefon Diggs", "Travis Etie…
  3  => Any["Jalen Hurts", "Jonathan Taylor", "Chris Olave", "Bryce Young", "De…
  1  => Any["Josh Allen", "Jaylen Waddle", "AJ Brown", "Nick Chubb", "Jaxon Smi…

In [430]:
pv = 0.90
first_choice, roster = dynasty_sf_model(5, pv);
second_choice, r2 = dynasty_sf_model(5, pv, [first_choice]);
third_choice, r3 = dynasty_sf_model(5, pv, [first_choice, second_choice])
first_choice, second_choice, third_choice

Total VORP: 236.1
Total VORP: 235.9
Total VORP: 235.8


("Devon Achane", "Zonovan Knight", "Lew Nichols")

In [432]:
roster

,Name,Position,Pick,ADP
,String,String,Int64,Float64
1,Justin Jefferson,WR,5,5.1
2,Travis Kelce,TE,16,21.4
3,Cooper Kupp,WR,25,31.5
4,Josh Jacobs,RB,36,39.1
5,Davante Adams,WR,45,45.7
6,Jerry Jeudy,WR,56,55.3
7,Joe Mixon,RB,65,73.7
8,Kenny Pickett,QB,76,81.7
9,Aaron Jones,RB,85,90.2


CURRENT SCRATCH

In [220]:
drafted = reset_draft();

In [208]:
just_drafted("Davante Adams", 9)

"Team 9 drafted Davante Adams!"

In [49]:
function redraft_model(team, temp=[])
    # create model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(model, "TimeLimit", 300)

    # PARAMETERS
    P = 321 # num_players
    min_qbs = 2
    max_qbs = 3
    min_rbs = 7
    max_rbs = 10
    min_wrs = 7
    max_wrs = 10
    min_tes = 2
    max_tes = 4

    opps = players_drafted_opps(team)

    num_picked = sum(opps[i, "Drafted"] for i in 1:P) + length(drafted[team])
    # for player in temp
    #     opps[player_to_index[player], "Drafted"] = 1
    # end

    picks = [team, 21-team]
    for i in 3:23
        push!(picks, picks[i-2]+20)
    end

    # VARIABLES
    @variable(model, x[i = 1:P], Bin) # whether or not player was drafted by desired team

    # OBJECTIVE FUNCTION
    @objective(model, Max, sum(data[i, "VORP"]*x[i] for i in 1:P))

    # CONSTRAINTS
    # roster must contain already drafted num_players
    @constraint(model, [i = drafted[team]], x[player_to_index[i]] == 1)

    # roster must not contain any opps
    @constraint(model, [i=1:P], x[i]+opps[i,"Drafted"] <= 1)

    # roster must not contain any temps
    @constraint(model, [i=1:length(temp)], x[player_to_index[temp[i]]] == 0)

    # roster must contain 23 players (no kicker/DEF)
    @constraint(model, sum(x[i] for i in 1:P) <= 23)

    # positional constraints
    @constraint(model, sum(x[player_to_index[i]] for i in qbs) >= min_qbs)
    @constraint(model, sum(x[player_to_index[i]] for i in qbs) <= max_qbs)
    @constraint(model, sum(x[player_to_index[i]] for i in rbs) >= min_rbs)
    @constraint(model, sum(x[player_to_index[i]] for i in rbs) <= max_rbs)
    @constraint(model, sum(x[player_to_index[i]] for i in wrs) >= min_wrs)
    @constraint(model, sum(x[player_to_index[i]] for i in wrs) <= max_wrs)
    @constraint(model, sum(x[player_to_index[i]] for i in tes) >= min_tes)
    @constraint(model, sum(x[player_to_index[i]] for i in tes) <= max_tes)


    # DRAFT POSITION CONSTRAINTS
    #at least 1 pick from set of players after each round
    for pick in 1:23
        @constraint(model, sum(x[i] for i in (picks[pick]-sum(opps[j, "Drafted"] for j in picks[pick]:P)):P) >= 24-pick)
    end

    # OPTIMIZE
    # solvetime = @elapsed optimize!(model)
    optimize!(model)

    roster = DataFrame(Name = String[], Pick = Int64[], ADP = Float64[], VORP = Float64[])
    count = 1
    for i in 1:P
        if value.(x[i]) == 1
            push!(roster, [data[i, "Name"], picks[count], data[i, "DynastySF_ADP"], data[i, "VORP"]])
            count += 1
        end
    end
    println("Total VORP: " * string(round(objective_value(model), digits = 1)))
    return roster
end

redraft_model (generic function with 2 methods)

In [ ]:
function dynasty_draft()
    # create model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(model, "TimeLimit", 300)

    # PARAMETERS
    T = 10 # teams
    R = 25 # roster spots
    P = 500 # num_players -- CHANGE
    min_qbs = 2
    max_qbs = 2
    min_rbs = 4
    max_rbs = 6
    min_wrs = 4
    max_wrs = 6
    min_tes = 2
    max_tes = 3

    # VARIABLES
    @variable(model, x[i = 1:T, j = 1:R] > 0, Int) # R roster spots each for N teams

    # OBJECTIVE FUNCTION - distribution for total expected value - maybe maximize the 25th percentile outcome?
    @objective(model, Max, sum(x[i,j] for i in 1:T, j in 1:R))

    # CONSTRAINTS
    # no player can be on two teams
    @constraint(model, [p = 1:P], sum(x[i,j]==p for i in 1:T, j in 1:R) <= 1)

    # position constraints for each team
    @constraint(model, [i = 1:T], sum(in(x[i,j], qbs) for j in 1:R) >= min_qbs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], qbs) for j in 1:R) <= max_qbs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], rbs) for j in 1:R) >= min_rbs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], rbs) for j in 1:R) <= max_rbs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], wrs) for j in 1:R) >= min_wrs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], wrs) for j in 1:R) <= max_wrs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], tes) for j in 1:R) >= min_tes)
    @constraint(model, [i = 1:T], sum(in(x[i,j], tes) for j in 1:R) <= max_tes)

    # already picked constraints
    @constraint(model, [i = 1:T], sum(x[i,j] for j in 1:R) == 16)


    # OPTIMIZE
    # solvetime = @elapsed optimize!(model)
    optimize!(model)


    return value.(x)
end